# Capture images from a camera, identify faces.

This runs on JupyterLabs so displaying the image remotely is a little clunky.  Would perform much better if you run the code natively on the device and then create a stream through HTML to remote viewer.

Notes:
* Install python3-matplotlib and cv2 via apt-get.  May need to further add pip3 install matplotlib in virtual terminal.
* Install face_recognition in vertual_env (pip3 install face_recognition)
* Install tools on the pi to work out what the camera is: 'apt-get install v4l-utils fswebcam' then 'v4l2-ctl --list-devices' and 'fswebcam -r 1280x720 --no-banner --device /dev/device0 /images/image1.jpg'.
* Check permissions on video group for the user running the notebook.  usermod -a -G video <username>.
* Inside of Jupyter you cannot use imshow from cv2 if running remote (e.g. JupyterLab) since cv2.imshow() launches locally to the kernel.  Use mathplotlib instead
* To take this further and to match faces to live stream images and then label them then look at https://github.com/ageitgey/face_recognition/blob/master/examples/facerec_from_webcam_faster.py
  

In [1]:
# Imports
import cv2
import sys
import numpy
from matplotlib import pyplot as plt
from IPython.display import display, Image
import ipywidgets as widgets
import threading

# Set the debug flag to push more information out to console
debug = True
flip_image = False

In [2]:
# Create a Stop Button for use with display video data
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

In [3]:
# Create the method for running video capture and displaying button.  Only process every other frame.

def view(stop_button):
    if(debug): print(f'DEBUG: Capturing video and setting resolution')

    # Set a variable to alternate frame processing
    process_this_frame = True

    # Start a capture and set the size
    capture = cv2.VideoCapture(0, apiPreference=cv2.CAP_V4L2)
    capture.set(3,640) # set Width
    capture.set(4,480) # set Height
    
    display_handle=display(None, display_id=True)
    
    while(True):
        # Start capturing
        return_val, captured_frame = capture.read()

        if (process_this_frame):
            # Flip camera to be vertical otherwise set the captured frame as the one to be processed
            if (flip_image):
                frame = cv2.flip(captured_frame,-1)
            else:
                frame = captured_frame

            # Convert to greyscale image for the image classifier to process
            grey_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 
                                                    "haarcascade_frontalface_default.xml")
            face_locations = face_classifier.detectMultiScale(grey_frame, 
                                                              scaleFactor=1.1, 
                                                              minNeighbors=5, 
                                                              minSize=(40, 40))

            # Where we have identified a face then draw a box around it
            for (top, right, bottom, left) in face_locations:
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Convert to RGB from BGR and then display it
            rgb_convert_result, rgb_frame = cv2.imencode('.jpeg', cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            display_handle.update(Image(data=rgb_frame.tobytes()))

            # Test the STOP button status
            if stopButton.value==True:
                if(debug): print(f'Stop button pressed, value {stopButton.value}')
                capture.release()
                display_handle.update(None)
                cv2.destroyAllWindows()
                
        # Make sure we do not process the next frame
        process_this_frame = not process_this_frame

In [ ]:
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()